# Bubble and Quick sort

## Utility

You would to skip it first.

In [96]:
import random
import operator
import timeit


idx = operator.itemgetter(0)
val = operator.itemgetter(1)


def pairs(arr):
    return zip(enumerate(arr), enumerate(arr[1:], 1))


def is_bad(pair):
    a, b = pair
    return val(a) > val(b)


def bad_pairs(arr):
    return filter(is_bad, pairs(arr))


def is_sorted(arr):
    return not any(bad_pairs(arr))


def move(arr, at, to):
    arr.insert(to, arr.pop(at))

    
def test_array():
    return [random.randint(0, 100) for _ in range(25)]


def time(sorting_function):
    return timeit.timeit(f'{sorting_function.__name__}(test_array())', number=2000, globals=globals())


def print_sorting(sorting_function):
    print(f'{sorting_function.__name__} ({time(sorting_function)} sec)')
    a = test_array()
    print(a)
    assert not is_sorted(a)
    sorting_function(a)
    assert is_sorted(a)
    print(a)

## [Bubble sort](#bubble_sort)

In [97]:
def bubble_sort(arr):
    while not is_sorted(arr):
        for a, b in bad_pairs(arr):
            move(arr, idx(a), idx(b))

## [Quick sort](#quick_sort)

### Partition scheme

All magic of sorting happens during partition process. We divide the array on two partitions: `[low .. part] <= [part .. high]`.

There are two most popular schemes of partition: Hoare and Lomuto. Lomuto's scheme is less efficient, but more simple.

#### Lomuto partition scheme

We choose the last array item `arr[high]` as **pivot**.

Then, we shifting low bound of array `low += 1` and compare the item `arr[low]` with **pivot**. If `pivot < item`, we place the item after the pivot.

In [98]:
def partition_lomuto(arr, low, high):
    while low < high:
        if arr[high] < arr[low]:
            move(arr, low, high)
            high -= 1
        else:
            low += 1
    return high

### Main function

This is Quick sort entry point. We recursively repeating partition function.

In [99]:
def quick_sort(arr, low=0, high=-1):
    high += 0 if 0 <= high else len(arr)
    if low < high:
        part = partition_lomuto(arr, low, high)
        quick_sort(arr, low, part - 1)
        quick_sort(arr, part + 1, high)

## [Testing](#testing)

In [100]:
print_sorting(bubble_sort)
print()
print_sorting(quick_sort)

bubble_sort (1.1741696240005695 sec)
[19, 27, 25, 43, 51, 34, 92, 41, 19, 61, 24, 59, 61, 1, 65, 25, 91, 13, 46, 10, 1, 59, 76, 48, 40]
[1, 1, 10, 13, 19, 19, 24, 25, 25, 27, 34, 40, 41, 43, 46, 48, 51, 59, 59, 61, 61, 65, 76, 91, 92]

quick_sort (0.3513152559989976 sec)
[3, 23, 100, 96, 18, 37, 45, 2, 77, 63, 21, 9, 70, 38, 47, 67, 86, 19, 6, 97, 87, 88, 30, 10, 32]
[2, 3, 6, 9, 10, 18, 19, 21, 23, 30, 32, 37, 38, 45, 47, 63, 67, 70, 77, 86, 87, 88, 96, 97, 100]
